In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DistilBertTokenizer


In [ ]:
#t=pd.read_csv('/content/drive/MyDrive/Hotel_Reviews/TAJ_reviews_only.csv')
t=pd.read_csv('/content/drive/MyDrive/Hotel_Reviews/TAJ_reviews_only_with_labels.csv')

In [ ]:
t
t=t.drop(['Unnamed: 0'],axis=1)

In [ ]:
t

,Review,Hotel_Name,room,staff,service,food,clean,location
0,"I recently stayed at Taj Skyline, and it was a...",Taj,1,1,1,1,1,1
1,Taj Skyline Hotel offers excellent service wit...,Taj,0,1,1,1,0,0
2,Location: Outside the city but easy connectivi...,Taj,1,1,1,1,0,1
3,I recently had a very disappointing experience...,Taj,0,0,-1,0,0,0
4,Services at its best 😊\n\nHad the opportunity ...,Taj,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
203,Great place,Taj,0,0,0,0,0,1
204,Nice hotel nd rod,Taj,0,0,0,0,0,0
205,Amazing,Taj,0,0,0,0,0,0
206,On,Taj,0,0,0,0,0,0


In [ ]:
def truncate_reviews(df, review_column, max_length=512):
    """
    Truncate reviews in the dataframe that exceed the maximum token length of 512 tokens.

    Args:
        df (pd.DataFrame): The DataFrame containing the reviews.
        review_column (str): The name of the column containing reviews.
        max_length (int): The maximum number of tokens allowed (default: 512).

    Returns:
        pd.DataFrame: DataFrame with truncated reviews where applicable.
    """
    # Initialize the DistilBERT tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    # Function to tokenize and truncate review if its token length exceeds max_length
    def truncate_review(review):
        # Tokenize the review and check its token length
        tokens = tokenizer.tokenize(review)
        if len(tokens) > max_length:
            # Truncate the tokens to max_length
            truncated_tokens = tokens[:max_length]
            # Convert tokens back to string
            truncated_review = tokenizer.convert_tokens_to_string(truncated_tokens)
            return truncated_review
        else:
            # Return the review as is if it does not exceed max_length
            return review

    # Apply the truncate function to the review column
    df[review_column] = df[review_column].apply(truncate_review)

    return df

In [ ]:
def clean_review(review):
    # List of words to replace
    clean_list = ['clean', 'cleaned', 'cleaner', 'cleaners', 'cleaniness', 'cleaning', 'cleanliness',
       'cleanly', 'cleanness', 'cleans', 'cleant', 'cleanup', 'reclean', 'roomcleanliness',
       'unclean', 'uncleaned', 'veryclean']
    room_list=['rooms','room5','inroom','roomall','roombut', 'roomcolour', 'roomi', 'roomroom', 'roomthe','roomy','yroom']
    # comfortable_list=['comfort','comfortable', 'comfortableall', 'comfortablegood', 'comfortableneat', 'comfortableroom', 'comfortablestaff', 'comfortablethank', 'comfortablethe', 'comfortably', 'comfortbale', 'comforter', 'comforting', 'comforts']

# Sample review text (replace this with your actual text data)

# Loop through the list and replace variations with the base word "clean"
    for word in clean_list:
      review = review.replace(word, "clean")

    for i in room_list:
      review=review.replace(i,'room')


    review = review.lower()  # Convert to lowercase
    review = review.replace("rooms", "room")  # Replace "rooms" with "room"
    review = review.replace("services", "service")  # Replace "services" with "service"
    review = review.replace("experiences", "experience")
    review = review.replace("food.", "food")
    review = review.replace("foods", "food")
    review = review.replace("service.", "service")
    review = review.replace("stay.", "stay")
    review = review.replace("staff.", "staff")
    review = review.replace("room.", "room")
    review = review.replace("location.", "location")
    review = review.replace("experience.", "experience")
    review = review.replace("room,", "room")
    review = review.replace("service,", "serivce")
    review = review.replace("staff,", "staff")
    review = review.replace("food,", "food")
    review = review.replace("serivce", "service")
    review = review.replace("located", "location")
    review = review.replace("locations", "location")
    review = review.replace("staying", "stay")
    review=review.replace("clean,","clean")
    review=review.replace("clean.","clean")
    review=review.replace("stay,","stay")
    review=review.replace("breakfast.","breakfast")
    review=review.replace("breakfast,","breakfast")
    review=review.replace("staffs,","staff")

    review=review.replace("comfortable,","comfortable")
    review=review.replace("comfortable.","comfortable")





     # Replace "experiences" with "experience"
    return review

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

def create_vocabulary(dataframe, column_name):
    # Combine all reviews into a single text
    all_reviews = " ".join(dataframe[column_name].astype(str))

    # Use CountVectorizer to tokenize and extract unique words
    vectorizer = CountVectorizer()
    vectorizer.fit([all_reviews])

    # Get the vocabulary
    vocabulary = sorted(vectorizer.get_feature_names_out())
    return vocabulary

In [ ]:
taj=truncate_reviews(t,'Review')
#novotel=truncate_reviews(nov,'Review')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Apply the function to create the "CleanedReview" column
taj["CleanedReview"] = taj["Review"].apply(clean_review)
#novotel["CleanedReview"] = novotel["Review"].apply(clean_review)

In [ ]:
taj

,Review,Hotel_Name,room,staff,service,food,clean,location,CleanedReview
0,"I recently stayed at Taj Skyline, and it was a...",Taj,1,1,1,1,1,1,"i recently stayed at taj skyline, and it was a..."
1,Taj Skyline Hotel offers excellent service wit...,Taj,0,1,1,1,0,0,taj skyline hotel offers excellent service wit...
2,Location: Outside the city but easy connectivi...,Taj,1,1,1,1,0,1,location: outside the city but easy connectivi...
3,I recently had a very disappointing experience...,Taj,0,0,-1,0,0,0,i recently had a very disappointing experience...
4,Services at its best 😊\n\nHad the opportunity ...,Taj,0,0,1,0,0,0,service at its best 😊\n\nhad the opportunity t...
...,...,...,...,...,...,...,...,...,...
203,Great place,Taj,0,0,0,0,0,1,great place
204,Nice hotel nd rod,Taj,0,0,0,0,0,0,nice hotel nd rod
205,Amazing,Taj,0,0,0,0,0,0,amazing
206,On,Taj,0,0,0,0,0,0,on


In [ ]:
taj_vocabulary = create_vocabulary(taj, 'CleanedReview')

# Print the size of the vocabulary and first few unique words
print(f"Vocabulary Size: {len(taj_vocabulary)}")

Vocabulary Size: 2244


In [ ]:
from collections import Counter
from nltk.corpus import stopwords
import nltk

# Download the stop words if not already available
nltk.download('stopwords')

def count_word_frequencies_no_stopwords(dataframe, column_name):
    # Combine all reviews into a single text
    all_reviews = " ".join(dataframe[column_name].astype(str))

    # Tokenize the text into words
    words = all_reviews.split()

    # Load English stop words
    stop_words = set(stopwords.words('english'))

    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Count the frequency of each word
    word_frequencies = Counter(filtered_words)

    # Get the 20 most frequent words
    most_frequent_words = word_frequencies.most_common(40)

    return word_frequencies, most_frequent_words

# Example usage
word_frequencies, most_frequent_words = count_word_frequencies_no_stopwords(taj, 'CleanedReview')

# Print the 20 most frequent words
print("Top 40 Most Frequent Words (Without Stop Words):")
for word, freq in most_frequent_words:
    print(f"{word}: {freq}")


Top 40 Most Frequent Words (Without Stop Words):
taj: 144
room: 137
service: 110
staff: 95
.: 91
experience: 86
stay: 84
hotel: 84
good: 77
food: 76
skyline: 61
,: 56
breakfast: 45
one: 44
also: 39
best: 36
like: 34
place: 33
ahmedabad: 33
clean: 31
us: 30
check: 29
location: 28
amazing: 28
excellent: 27
everything: 26
great: 26
restaurant: 25
would: 24
nice: 24
time: 23
comfortable: 22
dining: 21
chef: 21
say: 20
mr: 20
team: 20
highly: 18
luxurious: 18
-: 18


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
ground_truth=t[['room','staff','service','food','clean','location']]


In [ ]:
ground_truth

,room,staff,service,food,clean,location
0,1,1,1,1,1,1
1,0,1,1,1,0,0
2,1,1,1,1,0,1
3,0,0,-1,0,0,0
4,0,0,1,0,0,0
...,...,...,...,...,...,...
203,0,0,0,0,0,1
204,0,0,0,0,0,0
205,0,0,0,0,0,0
206,0,0,0,0,0,0


In [ ]:
taj2 = taj[['CleanedReview']]


In [ ]:
taj2

,CleanedReview
0,"i recently stayed at taj skyline, and it was a..."
1,taj skyline hotel offers excellent service wit...
2,location: outside the city but easy connectivi...
3,i recently had a very disappointing experience...
4,service at its best 😊\n\nhad the opportunity t...
...,...
203,great place
204,nice hotel nd rod
205,amazing
206,on


In [ ]:
# Load pre-trained model and tokenizer
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
absa_model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [ ]:
def analyze_aspects(review, aspect_groups):
    """
    Analyzes sentiment for each aspect group in a review.

    Args:
        review (str): The review text.
        aspect_groups (dict): A dictionary of aspect groups with their associated keywords.

    Returns:
        dict: A dictionary containing sentiment scores (positive/negative) for each aspect group.
        dict: A dictionary containing aspect classification: 1 (positive), -1 (negative), 0 (not present).
    """
    results = {}
    aspect_classification = {}

    for aspect, keywords in aspect_groups.items():
        # Check if any keyword for the aspect is mentioned in the review
        if any(keyword.lower() in review.lower() for keyword in keywords):
            # Format the input for ABSA using the first matching keyword
            matching_keyword = next(keyword for keyword in keywords if keyword.lower() in review.lower())
            formatted_input = f"[CLS] {review} [SEP] {matching_keyword} [SEP]"
            inputs = absa_tokenizer(formatted_input, return_tensors="pt").to(device)

            # Get model predictions
            with torch.no_grad():
                outputs = absa_model(**inputs)

            # Apply softmax to calculate probabilities for sentiment classes
            probs = F.softmax(outputs.logits, dim=1).cpu().numpy()[0]

            # Store sentiment scores
            #results[f"{aspect}_negative"] = probs[0]
            results[f"{aspect}_positive"] = probs[2]  # Assuming class 2 is "positive"

            # Determine aspect classification based on the condition
            if probs[2] > 0.6:  # Positive probability > 0.6
                aspect_classification[aspect] = 1
            else:  # Otherwise, classify as negative
                aspect_classification[aspect] = -1
        else:
            # Aspect not present in the review
            aspect_classification[aspect] = 0

    return results, aspect_classification


In [ ]:
import pandas as pd
from tqdm import tqdm  # To track progress

# Define aspect groups
aspect_groups = {
    "food": ["food", "breakfast", "buffet", "dinner", "dining", "dishes", "cuisine", "lunch"],
    "clean": ["clean", "cleanliness", "hygiene"],
    "staff": ["staff", "attendant", "employees", "chef", "servers", "waiter", "manager", "team",
              "receptionist", "management", "housekeeping", "house-keeping"],
    "room": ["room", "spacious", "bed", "bedroom", "beds", "rooms"],
    "location": ["location", "located", "place", "view", "distance", "accessibility"],
    "service": ["luxury", "comfort", "comfortable", "service", "hospitality", "hospitable"]
}

# Enable tqdm progress tracking
tqdm.pandas()

# Function to process each review
def process_review(cleaned_review):
    # Analyze aspects for the cleaned review
    _, aspect_classification = analyze_aspects(cleaned_review, aspect_groups)

    # Return a dictionary of aspect classifications
    return {aspect: aspect_classification.get(aspect, 0) for aspect in aspect_groups.keys()}

# Apply the function to the CleanedReview column and create new columns
aspect_results = taj2["CleanedReview"].progress_apply(process_review)

# Convert the results (list of dictionaries) into a DataFrame
aspect_df = pd.DataFrame(list(aspect_results))

# Concatenate the aspect DataFrame with the original DataFrame
taj2 = pd.concat([taj2, aspect_df], axis=1)


100%|██████████| 208/208 [07:32<00:00,  2.17s/it]


In [ ]:
# predicted=pd.read_csv('/content/drive/MyDrive/Hotel_Reviews/TAJ_reviews_only_with_labels2222222222.csv')

In [ ]:
taj2

,CleanedReview,food,clean,staff,room,location,service
0,"i recently stayed at taj skyline, and it was a...",1,1,1,1,1,1
1,taj skyline hotel offers excellent service wit...,1,0,1,0,1,1
2,location: outside the city but easy connectivi...,1,0,1,1,1,1
3,i recently had a very disappointing experience...,0,0,-1,0,0,-1
4,service at its best 😊\n\nhad the opportunity t...,1,0,1,1,1,1
...,...,...,...,...,...,...,...
203,great place,0,0,0,0,1,0
204,nice hotel nd rod,0,0,0,0,0,0
205,amazing,0,0,0,0,0,0
206,on,0,0,0,0,0,0


In [ ]:
taj2.rename(columns={
    'food': 'predicted_food',
    'clean': 'predicted_clean',
    'staff': 'predicted_staff',
    'room': 'predicted_room',
    'location': 'predicted_location',
    'service': 'predicted_service'
}, inplace=True)


In [ ]:
predicted=taj2[['predicted_food','predicted_clean','predicted_staff','predicted_room','predicted_location','predicted_service']]

In [ ]:
# List of aspect names
aspects = ['food', 'clean', 'staff', 'room', 'location', 'service']

# Initialize a dictionary to store accuracy for each aspect
aspect_accuracies = {}

# Loop through each aspect to calculate accuracy
for aspect in aspects:
    # Get the predicted and ground truth columns
    predicted_column = f'predicted_{aspect}'
    ground_truth_column = aspect

    # Compare predicted and ground truth values for the aspect
    correct_predictions = (predicted2[predicted_column] == ground_truth[ground_truth_column]).sum()  # Correct matches
    total_predictions = predicted2[predicted_column].size  # Total number of predictions

    # Calculate accuracy for the aspect
    aspect_accuracy = correct_predictions / total_predictions
    aspect_accuracies[aspect] = aspect_accuracy

# Optionally, calculate average accuracy across all aspects
average_accuracy = sum(aspect_accuracies.values()) / len(aspect_accuracies)

# Print individual accuracies and average accuracy
for aspect, accuracy in aspect_accuracies.items():
    print(f"Accuracy for {aspect}: {accuracy:.2%}")
print(f"Average Accuracy: {average_accuracy:.2%}")


Accuracy for food: 87.50%
Accuracy for clean: 96.15%
Accuracy for staff: 88.94%
Accuracy for room: 91.83%
Accuracy for location: 87.50%
Accuracy for service: 91.35%
Average Accuracy: 90.54%


In [ ]:
# taj2.to_csv('/content/drive/MyDrive/Hotel_Reviews/TAJ_reviews_only_with_labels2222222222.csv')

In [ ]:
# Initialize lists to store the data
aspect_names = []
positive_count_list = []
negative_count_list = []
difference_list = []
positive_percentage_list = []

# Loop through the aspect columns
aspects = ['food', 'clean', 'staff', 'room', 'location', 'service']
for aspect in aspects:
    predicted_column = f'predicted_{aspect}'

    # Count occurrences of 1 (positive) and -1 (negative)
    positive_count = (taj2[predicted_column] == 1).sum()  # Count of positive reviews
    negative_count = (taj2[predicted_column] == -1).sum()  # Count of negative reviews
    #total_reviews = taj2[predicted_column].count()  # Total reviews for the aspect (including positive, negative, and neutral)

    # Calculate the difference
    difference = positive_count - negative_count
    total_reviews=positive_count + negative_count

    # Calculate the percentage of positive reviews
    positive_percentage = (positive_count /total_reviews ) * 100 if total_reviews > 0 else 0

    # Store the data
    aspect_names.append(aspect)
    positive_count_list.append(positive_count)
    negative_count_list.append(negative_count)
    difference_list.append(difference)
    positive_percentage_list.append(positive_percentage)

# Create a DataFrame to display the results
result_df = pd.DataFrame({
    'Aspect': aspect_names,
    'Positive Reviews': positive_count_list,
    'Negative Reviews': negative_count_list,
    'Difference': difference_list,
    'Percentage': positive_percentage_list
})


In [ ]:
result_df

,Aspect,Positive Reviews,Negative Reviews,Difference,Percentage
0,food,88,32,56,73.333333
1,clean,25,7,18,78.125000
2,staff,86,17,69,83.495146
3,room,67,19,48,77.906977
4,location,54,13,41,80.597015
5,service,99,14,85,87.610619


#Textblob

In [ ]:
from textblob import TextBlob

In [ ]:
# Define aspect groups
aspect_groups = {
    "food": ["food", "breakfast", "buffet", "dinner", "dining", "dishes", "cuisine", "lunch"],
    "clean": ["clean", "cleanliness", "hygiene"],
    "staff": ["staff", "attendant", "employees", "chef", "servers", "waiter", "manager", "team",
              "receptionist", "management", "housekeeping", "house-keeping"],
    "room": ["room", "spacious", "bed", "bedroom", "beds", "rooms"],
    "location": ["location", "located", "place", "view", "distance", "accessibility"],
    "service": ["luxury", "comfort", "comfortable", "service", "hospitality", "hospitable"]
}


def analyze_aspects_textblob(review, aspect_groups):
    aspect_classification = {}
    for aspect, keywords in aspect_groups.items():
        # Check if any keyword related to the aspect is in the review
        if any(keyword in review.lower() for keyword in keywords):
            # Calculate sentiment polarity using TextBlob
            polarity = TextBlob(review).sentiment.polarity

            # Classify sentiment based on polarity
            if polarity > 0:  # Positive sentiment
                aspect_classification[aspect] = 1
            elif polarity < 0:  # Negative sentiment
                aspect_classification[aspect] = -1
            else:  # Neutral sentiment
                aspect_classification[aspect] = 0
        else:
            # Aspect not mentioned in the review
            aspect_classification[aspect] = 0
    return aspect_classification


In [ ]:
# Assume `taj2` contains the CleanedReview column
taj2['Aspect_Analysis_TextBlob'] = taj2['CleanedReview'].apply(lambda x: analyze_aspects_textblob(x, aspect_groups))


<ipython-input-21-dad0faf1bd60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taj2['Aspect_Analysis_TextBlob'] = taj2['CleanedReview'].apply(lambda x: analyze_aspects_textblob(x, aspect_groups))


In [ ]:
# Convert the dictionary in Aspect_Analysis_TextBlob into separate columns
aspect_columns = list(aspect_groups.keys())
for aspect in aspect_columns:
    taj2[f'predicted_{aspect}'] = taj2['Aspect_Analysis_TextBlob'].apply(lambda x: x.get(aspect, 0))


<ipython-input-23-7963dc5df650>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taj2[f'predicted_{aspect}'] = taj2['Aspect_Analysis_TextBlob'].apply(lambda x: x.get(aspect, 0))
<ipython-input-23-7963dc5df650>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taj2[f'predicted_{aspect}'] = taj2['Aspect_Analysis_TextBlob'].apply(lambda x: x.get(aspect, 0))
<ipython-input-23-7963dc5df650>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [ ]:
taj2

,CleanedReview,Aspect_Analysis_TextBlob,predicted_food,predicted_clean,predicted_staff,predicted_room,predicted_location,predicted_service
0,"i recently stayed at taj skyline, and it was a...","{'food': 1, 'clean': 1, 'staff': 1, 'room': 1,...",1,1,1,1,1,1
1,taj skyline hotel offers excellent service wit...,"{'food': 1, 'clean': 0, 'staff': 1, 'room': 0,...",1,0,1,0,1,1
2,location: outside the city but easy connectivi...,"{'food': 1, 'clean': 0, 'staff': 1, 'room': 1,...",1,0,1,1,1,1
3,i recently had a very disappointing experience...,"{'food': 0, 'clean': 0, 'staff': -1, 'room': 0...",0,0,-1,0,0,-1
4,service at its best 😊\n\nhad the opportunity t...,"{'food': 1, 'clean': 0, 'staff': 1, 'room': 1,...",1,0,1,1,1,1
...,...,...,...,...,...,...,...,...
203,great place,"{'food': 0, 'clean': 0, 'staff': 0, 'room': 0,...",0,0,0,0,1,0
204,nice hotel nd rod,"{'food': 0, 'clean': 0, 'staff': 0, 'room': 0,...",0,0,0,0,0,0
205,amazing,"{'food': 0, 'clean': 0, 'staff': 0, 'room': 0,...",0,0,0,0,0,0
206,on,"{'food': 0, 'clean': 0, 'staff': 0, 'room': 0,...",0,0,0,0,0,0


In [ ]:
predicted_blob=taj2[['predicted_food','predicted_clean','predicted_staff','predicted_room','predicted_location','predicted_service']]

In [ ]:
# List of aspect names
aspects = ['food', 'clean', 'staff', 'room', 'location', 'service']

# Initialize a dictionary to store accuracy for each aspect
aspect_accuracies = {}

# Loop through each aspect to calculate accuracy
for aspect in aspects:
    # Get the predicted and ground truth columns
    predicted_column = f'predicted_{aspect}'
    ground_truth_column = aspect

    # Compare predicted and ground truth values for the aspect
    correct_predictions = (predicted_blob[predicted_column] == ground_truth[ground_truth_column]).sum()  # Correct matches
    total_predictions = predicted_blob[predicted_column].size  # Total number of predictions

    # Calculate accuracy for the aspect
    aspect_accuracy = correct_predictions / total_predictions
    aspect_accuracies[aspect] = aspect_accuracy

# Optionally, calculate average accuracy across all aspects
average_accuracy = sum(aspect_accuracies.values()) / len(aspect_accuracies)

# Print individual accuracies and average accuracy
for aspect, accuracy in aspect_accuracies.items():
    print(f"Accuracy for {aspect}: {accuracy:.2%}")
print(f"Average Accuracy: {average_accuracy:.2%}")


Accuracy for food: 84.13%
Accuracy for clean: 94.71%
Accuracy for staff: 87.50%
Accuracy for room: 94.71%
Accuracy for location: 88.46%
Accuracy for service: 90.38%
Average Accuracy: 89.98%


#DEmo

In [ ]:
def analyze_aspects(review, aspects):
    """
    Analyzes sentiment for each aspect in a review.

    Args:
        review (str): The review text.
        aspects (list): List of aspects to analyze.

    Returns:
        dict: A dictionary containing sentiment scores (positive/negative) for each aspect.
        dict: A dictionary containing aspect classification: 1 (positive), -1 (negative), 0 (not present).
    """
    results = {}
    aspect_classification = {}
    aspect_groups = {
    "food": ["food", "breakfast", "buffet", "dinner", "dining", "dishes", "cuisine", "lunch"],
    "clean": ["clean", "cleanliness", "hygiene"],
    "staff": ["staff", "attendant", "employees", "chef", "servers", "waiter", "manager", "team",
              "receptionist", "management", "housekeeping", "house-keeping", "team"],
    "room": ["room", "spacious", "bed", "bedroom", "beds", "rooms"],
    "location": ["location", "located", "place", "view", "distance", "accessibility"],
    "service": ["luxury", "comfort", "comfortable", "service", "hospitality", "hospitable"]
}


    for aspect in aspects:
        # Check if the aspect is mentioned in the review
        if aspect.lower() in review.lower():
            # Format the input for ABSA
            formatted_input = f"[CLS] {review} [SEP] {aspect} [SEP]"
            inputs = absa_tokenizer(formatted_input, return_tensors="pt").to(device)

            # Get model predictions
            with torch.no_grad():
                outputs = absa_model(**inputs)

            # Apply softmax to calculate probabilities for sentiment classes
            probs = F.softmax(outputs.logits, dim=1).cpu().numpy()[0]

            # Store sentiment scores
            results[f"{aspect}_negative"] = probs[0]
            results[f"{aspect}_positive"] = probs[2]  # Assuming class 2 is "positive"

            # Determine aspect classification
            if probs[2] > 0.6:  # Positive probability > 0.6
                aspect_classification[aspect] = 1
            else:  # Otherwise, classify as negative
                aspect_classification[aspect] = -1
        else:
            # Aspect not present in the review
            aspect_classification[aspect] = 0

    return aspect_classification


In [ ]:
# Example usage
review = "The rooms were really spacious and clean but service was not good.Staff was polite and helping.It is in good location with easy transportation facility"
aspects = ["staff", "food", "room", "service", "location", "clean"]

results = analyze_aspects(review, aspects)
print(results)


{'staff': 1, 'food': 0, 'room': 1, 'service': -1, 'location': 1, 'clean': 1}


In [ ]:
#from transformers import pipeline

# Initialize the sentiment analyzer pipeline
sentiment_analyzer = pipeline("text-classification", model="srimeenakshiks/aspect-based-sentiment-analyzer-using-bert")

# Define the review
review = "The rooms were really spacious and clean but service was not good. Staff was polite and helping. It is in good location with easy transportation facility."

# Define the aspects
aspects = ["staff", "food", "room", "service", "location", "clean"]

# Function to analyze aspects
def analyze_aspects(review, aspects):
    aspect_results = {}
    for aspect in aspects:
        # Use the pipeline to analyze the sentiment for each aspect
        result = sentiment_analyzer(review, aspect=aspect)
        # Extract the label and store it
        aspect_results[aspect] = result[0]["label"]  # 'LABEL_0', 'LABEL_1', etc. can be mapped to actual sentiment
    return aspect_results

# Get the results
results = analyze_aspects(review, aspects)
print(results)


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cpu


TypeError: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'aspect'

In [ ]:
from transformers import pipeline

# Initialize the sentiment analyzer pipeline
sentiment_analyzer = pipeline("text-classification", model="srimeenakshiks/aspect-based-sentiment-analyzer-using-bert")

# Define the review
review = "The rooms were really spacious and clean but service was not good. Staff was polite and helping. It is in a good location with easy transportation facility."

# Define the aspects
aspects = ["staff", "food", "room", "service", "location", "clean"]

# Function to analyze aspects
def analyze_aspects(review, aspects):
    aspect_results = {}
    for aspect in aspects:
        # Concatenate aspect and review for model input
        aspect_review = f"{review} [ASP] {aspect}"
        # Use the pipeline to analyze the sentiment for the aspect
        result = sentiment_analyzer(aspect_review)
        # Extract the sentiment label
        aspect_results[aspect] = result[0]["label"]  # Example labels: 'LABEL_0', 'LABEL_1', etc.
    return aspect_results

# Get the results
results = analyze_aspects(review, aspects)
print(results)


Device set to use cpu


{'staff': 'LABEL_0', 'food': 'LABEL_0', 'room': 'LABEL_0', 'service': 'LABEL_0', 'location': 'LABEL_0', 'clean': 'LABEL_0'}


In [ ]:
from transformers import pipeline

# Initialize the sentiment analyzer pipeline
sentiment_analyzer = pipeline("text-classification", model="srimeenakshiks/aspect-based-sentiment-analyzer-using-bert")

# Define the review and aspect
review = "The food was amazing, but the service was slow."
aspect = "service"

# Format the input text with the aspect
formatted_input = f"{review} [ASP] {aspect}"

# Perform sentiment analysis
result = sentiment_analyzer(formatted_input)
print(result)


Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.9900272488594055}]
